## Save a Copy This Notebook to Google Drive

(Ignore this step if you already did this step and are using the notebook stored in your Drive)

Save your own copy of the notebook in your Google Drive with the "Copy to Drive" button near the top left of the notebook. A copy of this notebook named "Copy of 02_Setup_do_once" will be saved to a folder called "Colab Notebooks" on your Drive. Now as you update the code, the changes will be saved. Otherwise, they would reset every time you closed the notebook. 

You can confirm you are using the copy of the notebook in your Drive by looking for the "Copy to Drive" button. If it's gone, you're in the right notebook.

## Connect this Google Colab Computer to your Drive

We need to connect the Google computer running the code to your Google Drive. (Which seems bizarre because this Colab notebook is already in your Drive - it's set up that way to be extra careful.) To connect the computer to your Drive, Google uses the the four "Open Authorization" (OAuth) steps:

1. Google will give you a link to let you ask to connect to your Drive.
2. Google will confirm you want to connect.
3. Google will give you a confirmation code.
4. You copy the code and paste it here to the computer running this code.

Because Colab notebooks give you a different computer each time, you'll need to run this step each time.

In [ ]:
# Run this cell to connect the notebook to your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Run this cell to bring in the concha code
!pip -q install concha
from concha import Planner

## Connect to Square

Concha learns from how each product sold to determine how much to make in the future. That history is saved by Square. Square only gives access to the level of detail you'll need to "developers" creating programs that connect to Square (e.g. QuickBooks, Mailchimp). The following steps show how to tell Square you are a developer to connect to your own data.

1.) Activate yourself as a developer here: https://developer.squareup.com/apps.

2.) Create an "app", you can call it anything you want.

3.) Click "Open" on the card shaped icon for the "Application" you just created.

4.) On the menu on left side, click on "OAuth".

5.) Set "Production Redirect URL" to "http://www.google.com"

6.) Copy and paste the "Production Application ID" and the "Production Application Secret" (Click "Show"
    first) into the next cell.

The app you are creating is like a data "agent" to pull detailed records of exactly when each item was ordered each day. The "Application ID" and "Application Secret" are for the agent you made.

In the following cell 
you are granting the agent (limited) access to your own sales history. It requests access to read order histories ("ORDERS READ") and store locations ("MERCHANT PROFILE READ"). Setting the redirect URL to www.google.com
is just a trick to get the confirmation code Square creates once you grant access.

In [ ]:
#@title Square Application Information

production_application_id = "" #@param {type:"string"}
production_application_secret = "" #@param {type:"string"}

from concha.importers import Square
sqr = Square()
request_url = sqr.create_access_request_url(production_application_id, production_application_secret)
print("Copy this url into a browswer to request data access for the agent you created: \n")
print(request_url)

After going to the url printed out above, you should be redirected to the google homepage, except that the authorization code is now listed in the url. Copy the
new url, paste it below and run the cell to get the access token.

In [ ]:
#@title Square Application Information

redirect_url = "" #@param {type:"string"}
response = sqr.get_access_token(redirect_url)
response.body

If you see an "access_token" in the response, then it worked!

Why go through all this trouble to create an access token instead of just using the "Production Access Token provided on the developer website? Because the token we created is safer: it only allows access for reading orders and business locations. The one on the website gives access to everything.

The new Square credentials are stored on your drive in:
`/content/drive/My Drive/concha_planners/importers/square.json`

In [ ]:
# Now let's see the list of vendor locations.
locs = sqr.get_locations()
locs

## Weather

To look up the past weather conditions on each day, you need an API key for NOAA.

1.) Go here: https://www.ncdc.noaa.gov/cdo-web/token and put in your email.

2.) Enter the API key they send you in the next cell.

In [ ]:
#@title NOAA (Weather) API Key

noaa_api_key = "" #@param {type:"string"}

from concha.weather import NOAA
weather = NOAA()
weather.set_api_key(noaa_api_key)

## Check Everything

Let's make a planner to make sure it all worked.

In [ ]:
from concha import Planner
example = Planner(planner_name="example")
example.set_importer(sqr)
first_location = list(sqr.settings['locations'].keys())[0]
example.set_location(first_location)
example.set_weather_agent(weather)
example.set_weather_station()
forecast = example.generate_daily_forecast_metadata()
forecast

If `forecast` is the weather for the next six days at the specified location
then everything is working.